In [1]:
path_to_results_with_negative_values = '../20240620-gradcam-kernelshap-ig-128/imagenette2/IntegratedGradients'

In [2]:
path_to_copied_results = 'imagenette2/IntegratedGradients'

In [15]:
source_images = "../../datasets/imagenette2/train/"

In [3]:
# create path_to_copied
import os
os.makedirs(path_to_copied_results, exist_ok=True)

In [31]:
# copy all files from all folders recursively from path_to_results_with_negative_values to path_to_copied_results. 
# if filename ends with npy then load it, take abs and save
import shutil
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
from pytorch_grad_cam.utils.image import show_cam_on_image
from matplotlib.colors import LinearSegmentedColormap
import copy

total_length = sum(1 for _, _, _ in os.walk(path_to_results_with_negative_values))

for root, dirs, files in tqdm(os.walk(path_to_results_with_negative_values), total=total_length):
    for file in files:
        if file.endswith('.npy'):
            path_to_file = os.path.join(root, file)
            data = np.load(path_to_file)
            data = np.abs(data)
            path_to_file_copied = os.path.join(path_to_copied_results, path_to_file[len(path_to_results_with_negative_values)+1:])
            os.makedirs(os.path.dirname(path_to_file_copied), exist_ok=True)
            np.save(path_to_file_copied, data)
            # plt.imshow(plt.imread(os.path.join(source_images,*path_to_file.split('\\')[-2:]).replace('.npy','.JPEG')))
            
            path_to_original_image = os.path.join(source_images,*path_to_file.split('\\')[-2:]).replace('.npy','.JPEG')
            output_png_path = path_to_file_copied.replace('.npy', '.png')
            output_png_path_only_explanation = output_png_path.replace('.png', '_only_explanation.png')


            # create white-green image
            explanation = copy.deepcopy(data)
            cmap = LinearSegmentedColormap.from_list("white-green", ["white", "green"])
            vmin, vmax = 0, 1
            attribution_normalized = 255 * (explanation - vmin) / (vmax - vmin)
            attribution_normalized = attribution_normalized.astype(np.uint8)    
            attribution_colored = cmap(attribution_normalized / 255.0)[:, :, :3] 
            attribution_colored = (attribution_colored * 255).astype(np.uint8)     
            attribution_colored_bgr = cv2.cvtColor(attribution_colored, cv2.COLOR_RGB2BGR)    
            cv2.imwrite(output_png_path_only_explanation, attribution_colored_bgr)

            # create image with explanation on top of original image
            image = cv2.imread(path_to_original_image, 1)[:, :, ::-1]
            image = cv2.resize(image, (224, 224))
            image = np.float32(image) / 255
            image_with_explanation = show_cam_on_image(image, data)
            cv2.imwrite(output_png_path, image_with_explanation)


        if file.endswith('txt'):
            path_to_file = os.path.join(root, file)
            path_to_file_copied = os.path.join(path_to_copied_results, path_to_file[len(path_to_results_with_negative_values)+1:])
            os.makedirs(os.path.dirname(path_to_file_copied), exist_ok=True)
            shutil.copyfile(path_to_file, path_to_file_copied)

100%|██████████| 155/155 [00:51<00:00,  3.03it/s]
